### Setup your environnement variables

In [3]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

In [1]:
from dotenv import load_dotenv
assert load_dotenv("../.env.local") # Load environment variables from a .env file

# Get started

In [3]:
from uiform import UiForm
uiclient = UiForm()
models_list = uiclient.models.list()

# Example 1 - Analyze informations from a document

#### Option A - You use our preprocess endpoint and openAI client

In [5]:
from uiform.client import UiForm

uiclient = UiForm()

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="o3-mini",
    modality="text",
    temperature=0,
    reasoning_effort="low",
    image_resolution_dpi=96,
    browser_canvas="A4",
)

print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500, currency='EUR') client=ClientData(company_name='Acme Corporation', VAT_number='GB123456789', city='London', postal_code='WC2N 5DU', country='GB', code=None, email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='Acme Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number=None, email_address=None, pickup_datetime=PickupDatetimeData(date='2023-05-02', start_time='08:00:00', end_time='12:00:00'), observations='Pickup at Acme Corporation as per provided address in billing section.'), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+49 89 123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-05-03', start_time='10:00:00', en

In [7]:
completion.model_dump()

{'id': 'chatcmpl-BAxbTyV5rIjhiI41ghUaCsEXetppK',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"reasoning___root": "The document is a clear booking confirmation in French with sectioned details including client, transporter, loading site, billing address, shipment details, and pricing. A single shipment is described with explicit pickup and delivery details, goods specifications including dangerous goods and temperature requirements, as well as agreed transport price.", "booking_id": "SHIP-001", "payment": {"total_price": 1500, "currency": "EUR"}, "client": {"company_name": "Acme Corporation", "reasoning___VAT_number": "Found VAT number GB123456789 in the billing address section labelled \'TVA\'. Although a French TVA appears for the transporter, the billing address uses GB VAT which is more relevant for the client.", "VAT_number": "GB123456789", "city": "London", "postal_code": "WC2N 5DU", "country": "GB", "code": null, "email":

In [13]:
import json
from uiform import UiForm, Schema
from openai import OpenAI

with open("freight/schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = UiForm()
doc_msg = uiclient.documents.create_messages(
    document = "/Users/sachaichbiah/CUBE_local/monorepo/backend/data/freight_documents/BookingConfirmation.pdf",
    modality="text"
)

schema_obj = Schema(
    json_schema = json_schema
)


# Now you can use your favorite model to analyze your document
client = OpenAI()
completion = client.beta.chat.completions.parse(
    model="o3-mini", 
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.id,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    },
    reasoning_effort="low"
)

In [14]:
completion.model_dump()

{'id': 'chatcmpl-BAxgkNL6zHDXV4suMFdbMfZ5MjLog',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{\n  "reasoning___root": "This document is a booking confirmation (confirmation d\'affrètement) issued on 30/11/23 by TRANSPORTS DU VEXIN. It provides details for a single shipment with specified pickup and delivery instructions, pricing, and special instructions. Key parties include the issuer TRANSPORTS DU VEXIN (client/billing party) and the recipient GROUSSARD for the proof of contact information; however, the shipment itself is arranged from FORAGE MECANIQUE DE PRECISION (sender) to MAN ENERGY SOLUTIONS (recipient). The document specifies transport times, cargo details, pricing and payment terms, and several operational instructions. Some fields, like vehicle type and specific equipment needs, are not explicitly provided and have been left as null.",\n  "booking_id": "2311523D",\n  "payment": {\n    "total_price": 183.00,\n    "curr